# Recurrent Neural Networks

Grzegorz Statkiewicz, Mateusz Matukiewicz

## Overview

The structure of the direcotry should be as follows:

```
.
├── data
│   ├── train.pkl
│   └── test_no_target.pkl
└── main.ipynb
```



## Setup

Select the device to use

In [1]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [ ]:
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import numpy as np
import os
import random
from torch.nn.utils.rnn import pad_sequence


### Config for reproductivity

In [3]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Data preparation

Load the data

In [4]:
train_path = "data/train.pkl"

In [5]:
def load_data(file_path):
    """Loads data from a pickle file."""
    try:
        with open(file_path, "rb") as f:
            data = pickle.load(f)
        return data
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None


In [6]:
train_raw_data = load_data(train_path)

print(f"Loaded {len(train_raw_data)} training samples.")

Loaded 2939 training samples.


In [ ]:
# # test data later
# test_raw_data = load_data(test_path)


Print sample data

In [9]:
import random


idx = random.randint(0, len(train_raw_data) - 1)
print(f"Sample data: {train_raw_data[idx]}")

Sample data: (array([ 80.,  80.,  80., 158.,  92.,  92.,  93.,  12.,  12.,  12.,   0.,
         0.,  92.,  92.,  13.,  93., 119.,  12.,  13., 172.,  77.,  92.,
         0.,   0.,  92.,  80.,  80., 158.,  92.,  92.,  93.,  12.,  12.,
        12.,   0.,   0.,  92.,  92.,  13.,  93., 119.,  12.,  13., 172.,
        77.,  92.,   0.,   0.,   0.,  12., 112.,  12.,   7.,  92., 127.,
        60., 157., 172.,  80.,  80.,  80.,  92.,  92., 127.,  74.,   7.,
       172.,  80.,  92.,  92.,  80.,  80.]), 0)


### chords and composers

In [12]:
all_chords_train = Counter()
all_composers_train = Counter()

for sequence, composer in train_raw_data:
    all_chords_train.update(sequence)
    all_composers_train.update([composer])

PAD_TOKEN = "<PAD>"
UNK_TOKEN = "<UNK>"


# Rezerwujemy indeks 0 dla PAD_TOKEN i 1 dla UNK_TOKEN
chord_to_idx = {PAD_TOKEN: 0, UNK_TOKEN: 1}
current_idx = 2 # Zaczynamy indeksowanie rzeczywistych akordów od 2
for chord, _ in all_chords_train.most_common():
    if chord not in chord_to_idx: # Upewnij się, że nie nadpisujemy specjalnych tokenów
        chord_to_idx[chord] = current_idx
        current_idx += 1

idx_to_chord = {idx: chord for chord, idx in chord_to_idx.items()}


VOCAB_SIZE = len(chord_to_idx)
PAD_IDX = chord_to_idx[PAD_TOKEN] # = 0
UNK_IDX = chord_to_idx[UNK_TOKEN] # = 1

print(f"Vocabulary size (chords): {VOCAB_SIZE}")
print(f"PAD_IDX: {PAD_IDX}, UNK_IDX: {UNK_IDX}")

composer_to_idx = {composer: i for i, (composer, _) in enumerate(all_composers_train.most_common())}
idx_to_composer = {idx: composer for composer, idx in composer_to_idx.items()}
OUTPUT_DIM = len(composer_to_idx)

print(f"Number of composers (classes): {OUTPUT_DIM}")

Vocabulary size (chords): 184
PAD_IDX: 0, UNK_IDX: 1
Number of composers (classes): 5
